# Playing with Recommenders

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

## Load Data

In [2]:
item_file = "../input/talent.pkl"
item_records, COLUMN_LABELS, READABLE_LABELS, ATTRIBUTES = pickle.load(open(item_file, "rb"))
item_df = pd.DataFrame(item_records)[ATTRIBUTES + COLUMN_LABELS].fillna(value=0)
item_df.head()

,id,name,price,reactions,stars,joined,categories,in_13_reasons_why,in_90_day_fiance,in_actors,...,in_ufc,in_vanderpump_rules,in_venture_capitalists,in_viners,in_vlog_squad,in_voice_actors,in_winter_sports,in_writers,in_younow,in_youtubers
0,perezhilton,Perez Hilton,27.0,924,5.0,April 2018,"[Reality TV, Commentators, Featured]",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,andydick,Andy Dick,99.0,340,4.9,October 2018,"[Reality TV, Comedians, Featured, Actors]",0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tjlavin,TJ Lavin,80.0,291,5.0,February 2018,"[Reality TV, Riders, Featured, Extreme Sports,...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,carsonkressley,Carson Kressley,59.0,290,5.0,October 2018,"[Reality TV, Bravo, Stylists, Featured, Actors...",0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,riffraff,RiFF RAFF,75.0,402,4.7,December 2017,"[Rappers, Featured, Musicians]",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
ITEM_NAMES = item_df["id"].values
like_file = "../input/likes.pkl"
like_csr = pickle.load(open(like_file, "rb"))
like_mat = np.array(like_csr.todense())
like_df = pd.DataFrame(like_mat, columns=ITEM_NAMES)
like_df.head()

,perezhilton,andydick,tjlavin,carsonkressley,riffraff,chumlee,gilbertgottfried,icet,benhiggy,laturtle,...,chrisjaialex,voman,el_peego,thisannaisbananas,zachharper,johnoberg,zacpullam,kansasbowling,mattcirulnick,itsscaleb__
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
def nonzero_index_set(arr):
    """
    Returns a set of  indices corresponding to non-zero
    entries in a numpy array (or other list-like).
    """
    res = set()
    for i, val in enumerate(arr):
        if val > 0:
            res.add(i)
    return res


def mat_to_sets(mat):
    """
    Converts a numpy matrix into a list of sets of column
    indices corresponding to non-zero row entries.
    """
    return [nonzero_index_set(row) for row in mat]

In [54]:
s_train, s_test, s_input, s_hidden = pickle.load(open("../input/train_test_set.pkl", "rb"))
print(len(s_train), len(s_test), len(s_input), len(s_hidden))

4000 1000 1000 1000


In [55]:
s_items = mat_to_sets(item_df[COLUMN_LABELS].values)
print(len(s_items))

5392


## Evaluation Metrics

In [29]:
import ml_metrics


def mapk_score(recs_true, recs_pred, k=10):
    """
    Computes the mean average precision at k (MAP@K) of recommendations.
    MAP@K = mean AP@K score over all users
    AP@K = (1 / m) * sum from 1 to k of (precision at i * relevance of ith item)
    Where m is the number of items in a user's hidden set
    Where k is the number of items recommended to each user
    params:
        recs_true: list of sets of hidden items for each user
        recs_pred: list of lists of recommended items, with each list
        k: number of recommendations to use in top set
    returns:
        float, range [0, 1], though score of 1 will be impossible if
        recs_true includes users who have more than k hidden items
    """
    return ml_metrics.mapk(recs_true, recs_pred, k)

In [40]:
def uhr_score(recs_true, recs_pred, k=10):
    """
    Computes the user hit tate (UHR) score of recommendations.
    UHR = the fraction of users whose top list included at
    least one item also in their hidden set.
    params:
        recs_true: list of sets of hidden items for each user
        recs_pred: list of lists of recommended items, with each list
        k: number of recommendations to use in top set
    returns:
        float, range [0, 1]
    """
    if len(recs_true) != len(recs_pred):
        note = "Length of true list {} does not match length of recommended list {}."
        raise ValueError(note.format(len(recs_true), len(recs_pred)))
    scores = []
    for r_true, r_pred_orig in zip(recs_true, recs_pred):
        r_pred = list(r_pred_orig)[0:k]
        intersect = set(r_true).intersection(set(r_pred))
        scores.append(1 if len(intersect) > 0 else 0)
    return np.mean(scores)

## Similarity Measures

In [128]:
def jaccard_sim(u, v):
    """
    Computes the Jaccard similarity between sets u and v.
    sim = intersection(u, v) / union(u, v)
    params:
        u, v: sets to compare
    returns:
        float between 0 and 1, where 1 represents perfect
            similarity and 0 represents no similarity
    """
    intersection = len(u.intersection(v))
    union = len(u.union(v))
    zero = 1e-10
    # Add small value to denominator to avoid divide by zero
    sim = intersection / (union + zero)
    return sim

In [129]:
def cosine_sim(u, v):
    """
    Computes the Cosine similarity between sets u and v.
    sim = intersection(u, v) / sqrt(|u| * |v|)
    Where |s| is the number of items in set s
    params:
        u, v: sets to compare
    returns:
        float between 0 and 1, where 1 represents perfect
            similarity and 0 represents no similarity
    """
    intersection = len(u.intersection(v))
    mag_u = len(u)
    mag_v = len(v)
    zero = 1e-10
    # Add small value to denominator to avoid divide by zero
    sim = intersection / (np.sqrt(mag_u * mag_v) + zero)
    return sim

In [182]:
import ast, operator


bin_ops = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Mod: operator.mod
}


custom_vars = {
    "a": lambda s: len(s[0].intersection(s[1])),
    "b": lambda s: len(s[1]) - len(s[0].intersection(s[1])),
    "c": lambda s: len(s[0]) - len(s[0].intersection(s[1])),
    "IX": lambda s: len(s[0].intersection(s[1])),
    "UN": lambda s: len(s[0].union(s[1])),
    "EX": lambda s: len(s[0].union(s[1])) - len(s[0].intersection(s[1])),
    "U": lambda s: len(s[0]),
    "V": lambda s: len(s[1]),
    "Z": lambda s: 1e-10,
}


def arithmetic_eval(s):
    """
    Author: poke (StackOverflow)
    Link: https://stackoverflow.com/questions/20748202/valueerror-malformed-string-when-using-ast-literal-eval
    """
    node = ast.parse(s, mode='eval')

    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        elif isinstance(node, ast.Str):
            return node.s
        elif isinstance(node, ast.Num):
            return node.n
        elif isinstance(node, ast.BinOp):
            return bin_ops[type(node.op)](_eval(node.left), _eval(node.right))
        else:
            raise Exception('Unsupported type {}'.format(node))

    return _eval(node.body)


def eval_expr(raw_expr, u, v):
    expr = raw_expr
    for var in custom_vars:
        if var in expr:
            val_fn = custom_vars[var]
            val = val_fn((u, v))
            expr = expr.replace(var, str(val))
    res = arithmetic_eval(expr)
    return res


def custom_sim(raw_expr):
    """
    Creates a custom similarity function from an expression.
    params:
        raw_expr: the mathematical expression for similarity
            between sets u and v
    returns:
        sim_fn(u, v): custom similarity function between
            sets u and v

    The expression may use the operators: [+, -, *, /, %]
    
    The expresssion may use the following variables:
    
               set u
             | 1 | 0 |
           -----------
           1 | a | b |
    set v  -----------
           0 | c |

    a = number of items in both sets
    b = number of items only in set v
    c = number of items only in set u
    IX = intersection(u, v) = a
    UN = union(u, v) = a + b + c
    EX = exclusion(u, v) = b + c
    U = |u| = a + c
    V = |v| = a + b
    Z = zero = small value to avoid divide by zero
    """
    
    def calculate_sim(u, v):
        """
        Computes the custom similarity between sets u and v.
        params:
            u, v: sets to compare
        returns:
            float, custom similarity score
        """
        return eval_expr(raw_expr, u, v)
    
    return calculate_sim

In [183]:
a = set([1, 2, 3])
b = set([2, 3])
print(jaccard_sim(a, b))
print(custom_sim("IX / (UN + Z)")(a, b))

0.6666666666444444
0.6666666666444444


In [189]:
%%prun
for i in range(10):
    for a in s_items:
        for b in s_items[0:10]:
            jaccard_sim(a, b)

## Recommender Algorithms

In [68]:
def collaborative_filter(recs_train, recs_input, k=10, j=3, sim_fn=jaccard_sim):
    """
    Collaborative filtering recommender system.
    params:
        recs_train: list of sets of liked item indices for train data
        recs_input: list of sets of liked item indices for input data
        k: number of items to recommend for each user
        j: number of similar users to base recommendations on
        sim_fn(u, v): function that returns a float value representing
            the similarity between sets u and v
    returns:
        recs_pred: list of lists of tuples of recommendedations,
            each tuple has (item index, relevance score) with the list
            of tuples sorted in order of decreasing relevance
    """
    recs_pred = []
    for src in recs_input:
        users = []
        for vec in recs_train:
            sim = sim_fn(src, vec)
            if sim > 0:
                users.append((sim, vec))
        j_users = min(len(users), j)
        if j_users > 0:
            top_users = sorted(users, key=lambda p: p[0], reverse=True)
            sim_user_sets = [vec for (sim, vec) in top_users[0:j_users]]
            opts = dict()
            for user_set in sim_user_sets:
                for item in user_set:
                    if item not in src:
                        if item not in opts:
                            opts[item] = 0
                        opts[item] += 1
            ranks = []
            for item in opts:
                count = opts[item]
                score = count / len(sim_user_sets)
                ranks.append((item, score))
            top_ranks = sorted(ranks, key=lambda p: p[1], reverse=True)
            k_recs = min(len(top_ranks), k)
            recs = top_ranks[0:k_recs]
            recs_pred.append(recs)
        else:
            recs_pred.append([])
    return recs_pred

In [69]:
def content_filter(items_train, recs_input, k=10, sim_fn=jaccard_sim):
    """
    Content-based filtering recommender system.
    params:
        items_train: list of sets of non-zero attribute indices for items
        recs_input: list of sets of liked item indices for input data
        k: number of items to recommend for each user
        sim_fn(u, v): function that returns a float value representing
            the similarity between sets u and v
    returns:
        recs_pred: list of lists of tuples of recommendedations,
            each tuple has (item index, relevance score) with the list
            of tuples sorted in order of decreasing relevance
    """
    recs_pred = []
    for src in recs_input:
        sim_items = []
        for item_new_idx, item_new in enumerate(items_train):
            total_sim = 0
            for item_liked_idx in src:
                item_liked = items_train[item_liked_idx]
                sim = sim_fn(item_new, item_liked)
                total_sim += sim
            mean_sim = total_sim / len(src) 
            if mean_sim > 0:
                sim_items.append((item_new_idx, mean_sim))
        top_ranks = sorted(sim_items, key=lambda p: p[1], reverse=True)
        k_recs = min(len(top_ranks), k)
        recs = top_ranks[0:k_recs]
        recs_pred.append(recs)
    return recs_pred

## Results

In [147]:
n_pred = 10
s_input_sample = s_input[0:n_pred]
s_hidden_sample = s_hidden[0:n_pred]

In [148]:
print("Strategy: Collaborative")
print("Similarity: Jaccard")
user_recs = collaborative_filter(s_train, s_input_sample)
recs_pred = [[item for item, score in recs] for recs in user_recs]
print("MAP = {0:.3f}".format(mapk_score(s_hidden_sample, recs_pred, k=10)))
print("UHR = {0:.3f}".format(uhr_score(s_hidden_sample, recs_pred, k=10)))

Strategy: Collaborative
Similarity: Jaccard
MAP = 0.095
UHR = 0.600


In [149]:
print("Strategy: Collaborative")
print("Similarity: Cosine")
user_recs = collaborative_filter(s_train, s_input_sample, sim_fn=cosine_sim)
recs_pred = [[item for item, score in recs] for recs in user_recs]
print("MAP = {0:.3f}".format(mapk_score(s_hidden_sample, recs_pred, k=10)))
print("UHR = {0:.3f}".format(uhr_score(s_hidden_sample, recs_pred, k=10)))

Strategy: Collaborative
Similarity: Cosine
MAP = 0.091
UHR = 0.500


In [150]:
print("Strategy: Content-Based")
print("Similarity: Jaccard")
user_recs = content_filter(s_items, s_input_sample)
recs_pred = [[item for item, score in recs] for recs in user_recs]
print("MAP = {0:.3f}".format(mapk_score(s_hidden_sample, recs_pred, k=10)))
print("UHR = {0:.3f}".format(uhr_score(s_hidden_sample, recs_pred, k=10)))

Strategy: Content-Based
Similarity: Jaccard
MAP = 0.058
UHR = 0.300


In [152]:
print("Strategy: Content-Based")
print("Similarity: Cosine")
user_recs = content_filter(s_items, s_input_sample, sim_fn=cosine_sim)
recs_pred = [[item for item, score in recs] for recs in user_recs]
print("MAP = {0:.3f}".format(mapk_score(s_hidden_sample, recs_pred, k=10)))
print("UHR = {0:.3f}".format(uhr_score(s_hidden_sample, recs_pred, k=10)))

Strategy: Content-Based
Similarity: Cosine
MAP = 0.052
UHR = 0.300
